In [1]:
from agents import *
from random import randrange, seed, choice

# Mining For Gold
This program simulates the environment in a dark gold man. You have are programming a robot that has burrowed into a random location in the mine and it has to move around and collect as much gold as possible. Your objective is to fill in the agent program to help the robot accomplish its mission.

## The Environment
First, we define a Gold class as well as a graphical environment for the robot to explore, we then generate a random mine. Read over this code, but there is nothing that needs changing here. The environment is drawn as a grid. Black squares are walls, which the robot cannot pass through. The gold squares are squares containing gold and the grey squares are empty squares. Finally, when we add the robot, it will show up as a red square.

In [2]:
class Gold(Thing):
    """
    A chunk of gold, it does nothing! (But the Wumpus gold in agents is a singleton)
    so we need this one here.
    """
    pass

class MineEnvironment(GraphicEnvironment):
    def __init__(self):
        color = {"Miner": (255,0,0),
                 "Gold": (253, 208, 23),
                 "Wall": (44, 53, 57)}
        super().__init__(16, 16, True, color)
        self.perceptible_distance=0
        self.add_walls()

        # place obstacles
        for i in range(64):
            self.add_thing(Wall(), self.empty_square())
        
        # place gold
        for i in range(42):
            self.add_thing(Gold(), self.empty_square())
    
    def empty_square(self):
        world = self.get_world()
        done = False
        while not done:
            x = randrange(len(world))
            y = randrange(len(world[x]))
            done = len(world[x][y]) == 0
        return x,y

mine = MineEnvironment()
mine.reveal()

,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,


## The miner
Now we come to the star of the show! Our intrepid little miner robot. The robot enters the field on a random empty square. Its constructor sets up the robots program and sets its initial direction to facing to the right. The robot can perform the following operations:

- TurnLeft
- TurnRight
- Forward
- Grab

These actions are perfomed by returning them from the program function. The program function receives an array of percepts, which is an array of tuples consisting of the objects on the robot's current square, as well as its distance form the robot. Because the robot can only see its current square, the second part of the tuple will always be 0. An example print out of the percept list is shown:

```
[(<Gold>, 0), (<Miner>, 0)]
```

In addition to the percept sequence, there is also the variable `self.bump` which is true if the robot rolled into a wall on its last move. You can use this to "feel" the shape of your randomized mine.

Note how the Miner is always included in the percept list. This is because the miner is part of the environment too! If you attempt to "Grab", the environment makes sure this is something that can be grabbed by calling the "can_grab" function.

As you can see, the robot's strategy is not a good one. It just randomly selects an action. Your job is to make the robot work better by modifying the program function. Remember, the mine will be different every time your run the program so you need to make a general agent. Good luck!

In [3]:
class Miner(Agent):
    def __init__(self, program=None):
        super().__init__(self.program)
        self.direction = Direction("right")
        self.history=[]
        
    def can_grab(self, thing):
        return thing.__class__.__name__ == "Gold"
    
    def program(self, percept):
        #TODO: Do something better than random selected actions.
        return choice(['TurnRight', 'TurnLeft', 'Forward', 'Grab'])
        

miner = Miner()
mine.add_thing(miner, mine.empty_square())
mine.run(50)

,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,
,,,,,,,,,,,,,,,


## How did we do?
The code below displays the robot's final amount of collected gold.

In [4]:
print("Gold Collected: {}".format(len(miner.holding)))


Gold Collected: 1
